In [1]:
import pandas as pd
from tqdm import tqdm
from pyarrow import csv
import pyarrow as pa

import numpy as np

import pandas as pd
from tqdm import tqdm
from pyarrow import csv
import pyarrow as pa

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from catboost import CatBoostClassifier
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer, MinMaxScaler, StandardScaler
from sklearn.metrics import auc, accuracy_score, precision_score, recall_score, roc_curve, precision_recall_curve

In [2]:
def default_transform(df):
    return df

def read_chunk(name, chunk_id, opts=csv.ConvertOptions(), transform=default_transform, extra=1):
    chunk=None
    chunksize = extra*10 ** 7
    with csv.open_csv(
        name,
        convert_options = opts,
        read_options=csv.ReadOptions(
            use_threads=True,
            block_size=chunksize
        )) as reader:

        i=0
        for next_chunk in reader:
            if next_chunk is None:
                break
            chunk=next_chunk
            if i == chunk_id:
                return transform(chunk.to_pandas())
            i+=1
            
    return None

def prep_NF_UQ(df):
    features_to_remove = ['L4_SRC_PORT', 'L4_DST_PORT', 'Attack', 'L4_SRC_PORT', 'L4_DST_PORT']
    df.drop(columns=features_to_remove, axis=1, inplace=True) 
    df['mask'] = 1
    return df

def prep_ddos(df):
    features_to_remove = ['Unnamed: 0', 'Flow ID', ' Source IP', ' Source Port', ' Destination IP', ' Destination Port', ' Timestamp']
    df.drop(columns=features_to_remove, axis=1, inplace=True) 
    
    df = df.rename(columns=
                   {' Protocol': 'PROTOCOL', 
                   ' Total Fwd Packets': 'IN_PKTS', 
                   ' Total Backward Packets': 'OUT_PKTS',
                   'Total Length of Fwd Packets': 'IN_BYTES',
                   ' Total Length of Bwd Packets': 'OUT_BYTES',
                   'Fwd IAT Total': 'DURATION_IN',
                   'Bwd IAT Total': 'DURATION_OUT',
                   ' Label': 'Attack',
                   ' Flow Duration': 'FLOW_DURATION_MILLISECONDS',
                   }, errors="raise")
    
    df['PROTOCOL'] = df['PROTOCOL'].astype(np.int64)
    df['IN_PKTS'] = df['IN_PKTS'].astype(np.int64)
    df['OUT_PKTS'] = df['OUT_PKTS'].astype(np.int64)
    df['IN_BYTES'] = df['IN_BYTES'].astype(np.int64)
    df['OUT_BYTES'] = df['OUT_BYTES'].astype(np.int64)
    df['DURATION_IN']= (df['DURATION_IN']/1000000).astype(np.int64)
    df['DURATION_OUT']= (df['DURATION_OUT']/1000000).astype(np.int64)
    df['Label'] = 1
    df['mask'] = 2
    return df

In [3]:
cols = prep_NF_UQ(pd.read_parquet('NF-UNSW-NB15-V2.parquet')).columns
cols = np.unique(np.concatenate((cols, prep_ddos(read_chunk('01-12/DrDoS_UDP.csv', chunk_id=0)).columns), axis=0))

def append(dataset, df):
    df[cols[np.invert(np.isin(cols, df.columns))]] = 0
    return dataset.append(df, ignore_index=True)

In [4]:
cols.shape

(114,)

In [5]:
dataset = pd.DataFrame(columns=cols)

In [6]:
df = prep_NF_UQ(pd.read_parquet('NF-UNSW-NB15-V2.parquet'))

In [7]:
dataset = append(dataset, df)
del df

/tmp/ipykernel_27380/1702868408.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return dataset.append(df, ignore_index=True)


In [8]:
dataset

,ACK Flag Count,Active Max,Active Min,Active Std,Average Packet Size,Avg Bwd Segment Size,Avg Fwd Segment Size,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Header Length,...,SERVER_TCP_FLAGS,SHORTEST_FLOW_PKT,SRC_TO_DST_AVG_THROUGHPUT,SRC_TO_DST_SECOND_BYTES,SimillarHTTP,Subflow Fwd Packets,TCP_FLAGS,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,mask
0,0,0,0,0,0,0,0,0,0,0,...,16,52,3648000,456.0,0,0,24,0,7240,1
1,0,0,0,0,0,0,0,0,0,0,...,24,52,4656000,708.0,0,0,24,8688,8688,1
2,0,0,0,0,0,0,0,0,0,0,...,24,52,5576000,928.0,0,0,24,10136,10136,1
3,0,0,0,0,0,0,0,0,0,0,...,24,52,6480000,1148.0,0,0,24,11584,11584,1
4,0,0,0,0,0,0,0,0,0,0,...,24,52,7544000,1478.0,0,0,24,14480,13032,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1986740,0,0,0,0,0,0,0,0,0,0,...,27,52,8512000,1064.0,0,0,27,10136,10136,1
1986741,0,0,0,0,0,0,0,0,0,0,...,27,52,8512000,1064.0,0,0,27,10136,10136,1
1986742,0,0,0,0,0,0,0,0,0,0,...,26,52,7952000,994.0,0,0,26,10136,7240,1
1986743,0,0,0,0,0,0,0,0,0,0,...,27,52,32112000,4014.0,0,0,27,44888,14480,1


In [9]:
dataset.to_csv('single.csv')

In [10]:
from os import listdir
from os.path import isfile, join

files = ['./01-12','./03-11']
for file in files:
    for f in listdir(file):
        if f[0] == '.':
            continue
        filename=file+"/"+f
        dataset = append(dataset, prep_ddos(read_chunk(filename, 0, extra=1)))

/tmp/ipykernel_27380/1702868408.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return dataset.append(df, ignore_index=True)
/tmp/ipykernel_27380/1702868408.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return dataset.append(df, ignore_index=True)
/tmp/ipykernel_27380/1702868408.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return dataset.append(df, ignore_index=True)
/tmp/ipykernel_27380/1702868408.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return dataset.append(df, ignore_index=True)
/tmp/ipykernel_27380/1702868408.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a 

In [11]:
dataset.to_csv('double.csv')

In [12]:
dataset['Label']

0          0
1          0
2          0
3          0
4          0
          ..
2395268    1
2395269    1
2395270    1
2395271    1
2395272    1
Name: Label, Length: 2395273, dtype: object

In [13]:
benign = pd.read_csv('benign.csv')[['IN_PACKETS', 'IN_OCTETS', 'TCP_FLAGS', 'PROTO']]
benign = benign.rename(columns={'IN_PACKETS': 'IN_PKTS', 'IN_OCTETS': 'IN_BYTES', 'TCP_FLAGS': 'TCP_FLAGS', 'PROTO': 'PROTOCOL'})

benign_t = pd.read_csv('benign-1.csv')[['IN_PACKETS', 'IN_OCTETS', 'TCP_FLAGS', 'PROTO']]
benign_t = benign.rename(columns={'IN_PACKETS': 'IN_PKTS', 'IN_OCTETS': 'IN_BYTES', 'TCP_FLAGS': 'TCP_FLAGS', 'PROTO': 'PROTOCOL'})

benign = benign.append(benign_t, ignore_index=True)
del benign_t

benign_t = pd.read_csv('benign-2.csv')[['IN_PACKETS', 'IN_OCTETS', 'TCP_FLAGS', 'PROTO']]
benign_t = benign.rename(columns={'IN_PACKETS': 'IN_PKTS', 'IN_OCTETS': 'IN_BYTES', 'TCP_FLAGS': 'TCP_FLAGS', 'PROTO': 'PROTOCOL'})

benign = benign.append(benign_t, ignore_index=True)
del benign_t

benign_t = pd.read_csv('gathered_newer.csv')[['IN_PACKETS', 'IN_OCTETS', 'TCP_FLAGS', 'PROTO']]
benign_t = benign.rename(columns={'IN_PACKETS': 'IN_PKTS', 'IN_OCTETS': 'IN_BYTES', 'TCP_FLAGS': 'TCP_FLAGS', 'PROTO': 'PROTOCOL'})

benign = benign.append(benign_t, ignore_index=True)
del benign_t

/tmp/ipykernel_27380/1849531848.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  benign = benign.append(benign_t, ignore_index=True)
/tmp/ipykernel_27380/1849531848.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  benign = benign.append(benign_t, ignore_index=True)
/tmp/ipykernel_27380/1849531848.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  benign = benign.append(benign_t, ignore_index=True)


In [14]:
benign['Label'] = 0

In [15]:
dos = pd.read_csv('hping3-1.csv')[['IN_PACKETS', 'IN_OCTETS', 'TCP_FLAGS', 'PROTO']]
dos = dos.rename(columns={'IN_PACKETS': 'IN_PKTS', 'IN_OCTETS': 'IN_BYTES', 'TCP_FLAGS': 'TCP_FLAGS', 'PROTO': 'PROTOCOL'})

dos_t = pd.read_csv('hping3-2.csv')[['IN_PACKETS', 'IN_OCTETS', 'TCP_FLAGS', 'PROTO']]
dos_t = dos_t.rename(columns={'IN_PACKETS': 'IN_PKTS', 'IN_OCTETS': 'IN_BYTES', 'TCP_FLAGS': 'TCP_FLAGS', 'PROTO': 'PROTOCOL'})

dos = dos.append(dos_t, ignore_index=True)
del dos_t

dos['Label'] = 1
dos['Attack'] = 1

/tmp/ipykernel_27380/3005927574.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dos = dos.append(dos_t, ignore_index=True)


In [16]:
benign

,IN_PKTS,IN_BYTES,TCP_FLAGS,PROTOCOL,Label
0,2,536,0,17,0
1,2,784,0,17,0
2,3,1340,0,17,0
3,2,592,0,17,0
4,2,840,0,17,0
...,...,...,...,...,...
90267,4,271,24,6,0
90268,2,104,16,6,0
90269,9,4095,2,6,0
90270,9,4095,2,6,0


In [17]:
dos

,IN_PKTS,IN_BYTES,TCP_FLAGS,PROTOCOL,Label,Attack
0,1,71,0,17,1,1
1,1,71,0,17,1,1
2,1,71,0,17,1,1
3,1,71,0,17,1,1
4,1,71,0,17,1,1
...,...,...,...,...,...,...
1673,1,67,0,17,1,1
1674,1,67,0,17,1,1
1675,1,67,0,17,1,1
1676,1,67,0,17,1,1


In [18]:
dataset = append(dataset, benign)
del benign
dataset = append(dataset, dos)
del dos

/tmp/ipykernel_27380/1702868408.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[cols[np.invert(np.isin(cols, df.columns))]] = 0
/tmp/ipykernel_27380/1702868408.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[cols[np.invert(np.isin(cols, df.columns))]] = 0
/tmp/ipykernel_27380/1702868408.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a 

In [19]:
dataset.to_csv('triple.csv')

In [20]:
dataset

,ACK Flag Count,Active Max,Active Min,Active Std,Average Packet Size,Avg Bwd Segment Size,Avg Fwd Segment Size,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Header Length,...,SERVER_TCP_FLAGS,SHORTEST_FLOW_PKT,SRC_TO_DST_AVG_THROUGHPUT,SRC_TO_DST_SECOND_BYTES,SimillarHTTP,Subflow Fwd Packets,TCP_FLAGS,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,mask
0,0,0,0,0,0,0,0,0,0,0,...,16,52,3648000,456.0,0,0,24,0,7240,1
1,0,0,0,0,0,0,0,0,0,0,...,24,52,4656000,708.0,0,0,24,8688,8688,1
2,0,0,0,0,0,0,0,0,0,0,...,24,52,5576000,928.0,0,0,24,10136,10136,1
3,0,0,0,0,0,0,0,0,0,0,...,24,52,6480000,1148.0,0,0,24,11584,11584,1
4,0,0,0,0,0,0,0,0,0,0,...,24,52,7544000,1478.0,0,0,24,14480,13032,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2487218,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0,0
2487219,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0,0
2487220,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0,0
2487221,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0,0
